<a href="https://colab.research.google.com/github/CelikAbdullah/deep-learning-notebooks/blob/main/Computer%20Vision/models/Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from tensorflow import keras

# Inception v1

## The stem component

In [38]:
def stem(inputs):
  '''
  Build the stem component of Inception v1.
  It extracts the coarse-level features of the input.

  input: the input image
  '''

  # apply ZeroPadding so that the size of the input is preserved
  x = keras.layers.ZeroPadding2D(padding=3)(inputs)

  # apply a strided 7x7 convolution to extract features
  # it will generate 64 output feature maps
  x = keras.layers.Conv2D(filters=64,
                          kernel_size=7,
                          strides=2,
                          padding="valid",
                          activation="relu",
                          kernel_initializer="glorot_uniform"
                          )(x)

  # again, apply ZeroPadding
  x = keras.layers.ZeroPadding2D(padding=1)(x)

  # apply strided max pooling with a 3x3 window
  # the size of each feature map will be reduced by 75%
  x = keras.layers.MaxPooling2D(pool_size=3, strides=2)(x)

  # apply a 1x1 convolution
  x = keras.layers.Conv2D(filters = 64,
                          kernel_size=1,
                          padding="same",
                          activation="relu",
                          kernel_initializer="glorot_uniform")(x)

  # zero-padding
  x = keras.layers.ZeroPadding2D(padding=1)(x)

  # a 3x3 convolution; it will output 192 feature maps
  x = keras.layers.Conv2D(filters=192,
                          kernel_size=3,
                          strides=1,
                          padding="valid",
                          activation="relu",
                          kernel_initializer="glorot_uniform"
                          )(x)

  # zero-padding
  x = keras.layers.ZeroPadding2D(padding=1)(x)

  # apply strided max pooling with a 3x3 window
  # the size of each feature map will be reduced by 75%
  x = keras.layers.MaxPooling2D(pool_size=3, strides=2)(x)


  return x

## The learner component

### Auxiliary Classifier

In [39]:
def auxiliary(x, classes):
  '''
  Build the auxiliary classifier.

  x       : input to the auxiliary classifier
  classes : number of output classes
  '''

  # strided average pooling operation with a 5x5 window
  x = keras.layers.AveragePooling2D(pool_size=5, strides=3)(x)

  # a 1x1 convolution which will output 128 feature maps
  x = keras.layers.Conv2D(filters=128,
                          kernel_size=1,
                          padding="same",
                          activation="relu",
                          kernel_initializer="glorot_uniform"
                          )(x)

  # flatten the feature maps before passing them to the Dense layer
  x = keras.layers.Flatten()(x)

  # a fully connected layer with 1024 neurons
  x = keras.layers.Dense(units=1024,
                         activation="relu",
                         kernel_initializer="glorot_uniform")(x)

  # dropout layer for regularization
  x = keras.layers.Dropout(rate=0.7)(x)

  # softmax layer to output probabilities for each class
  output = keras.layers.Dense(units=classes,
                              activation="softmax",
                              kernel_initializer="glorot_uniform")(x)


  return output

### Inception Block

In [40]:
def inception_block(input, filters_1x1, filters_3x3, filters_5x5, filter_pool):
  '''
  Build the inception block/module.

  input : input to the inception block/module
  filters_1x1 : nr. of filters for the 1x1 conv branch of the inception module
  filters_3x3 : nr. of filters for the 3x3 conv branch of the inception model
  filters_5x5 : nr. of filters for the 5x5 conv branch of the inception module
  filters_pool: nr. of filters for the max pooling branch of the inception module
  '''

  # create the 1x1 conv branch
  branch_1x1 = keras.layers.Conv2D(filters=filters_1x1[0],
                                   kernel_size=1,
                                   strides=1,
                                   padding="same",
                                   activation='relu',
                                   kernel_initializer='glorot_uniform')(input)

  # create the 3x3 conv branch with the 1x1 bottleneck convolution that performs dimensionality reduction
  branch_3x3 = keras.layers.Conv2D(filters=filters_3x3[0],
                                   kernel_size=1,
                                   strides=1,
                                   padding="same",
                                   activation='relu',
                                   kernel_initializer='glorot_uniform')(input)

  branch_3x3 = keras.layers.ZeroPadding2D(padding=1)(branch_3x3)
  branch_3x3 = keras.layers.Conv2D(filters=filters_3x3[1],
                                   kernel_size=3,
                                   strides=1,
                                   padding="valid",
                                   activation='relu',
                                   kernel_initializer='glorot_uniform')(branch_3x3)

  # create the 5x5 conv branch with the 1x1 bottleneck convolution that performs dimensionality reduction
  branch_5x5 = keras.layers.Conv2D(filters=filters_5x5[0],
                                   kernel_size=1,
                                   strides=1,
                                   padding="same",
                                   activation='relu',
                                   kernel_initializer='glorot_uniform')(input)

  branch_5x5 = keras.layers.ZeroPadding2D(padding=1)(branch_5x5)
  branch_5x5 = keras.layers.Conv2D(filters=filters_5x5[1],
                                   kernel_size=3,
                                   strides=1,
                                   padding="valid",
                                   activation='relu',
                                   kernel_initializer='glorot_uniform')(branch_5x5)

  # create the max pooling branch with 1x1 linear projection for dimensionality expansion
  branch_pool = keras.layers.MaxPooling2D(pool_size=3,
                                          strides=1,
                                          padding="same")(input)

  branch_pool = keras.layers.Conv2D(filters=filter_pool[0],
                                    kernel_size=1,
                                    strides=1,
                                    padding='same',
                                    activation='relu',
                                    kernel_initializer='glorot_uniform')(branch_pool)


  # Concatenate the output feature maps of the branches
  x = keras.layers.Concatenate()([branch_1x1, branch_3x3, branch_5x5, branch_pool])

  return x


### Inception Group

In [41]:
def group(x, blocks, pooling=True, classes=1000):
  '''
  Build an Inception group.

  x       : input to the inception group
  blocks  : list of tuples containing nr. of filters for each block in the group
  pooling : flag indicating whether to end the group with max pooling
  classes : nr. of classes for auxiliary classifier
  '''

  # list to save the auxiliary outputs
  aux = []

  # build the inception blocks
  for block in blocks:

    # adding the auxiliary classifier
    if block is None:
      # save the auxiliary output
      aux.append(auxiliary(x,classes))

    else:
       # create inception block
       x = inception_block(x,
                           block[0],
                           block[1],
                           block[2],
                           block[3])


  # if pooling is True
  if pooling:
    # zero-padding
    x = keras.layers.ZeroPadding2D(padding=(1,1))(x)

    # max pooling
    x = keras.layers.MaxPooling2D(pool_size=3, strides=2)(x)


  return x, aux

Now, we can finally build the learner component of Inception v1:

In [42]:
def learner(x, classes):
  '''
  Build the learner component.

  x       : input to the learner
  classes : nr. of classes
  '''

  # a list to store outputs from the auxiliary classifiers
  aux = []

  # set the block specs for group 3
  blocks = [((64,),  (96,128),   (16, 32), (32,)),
            ((128,), (128, 192), (32, 96), (64,))]

  # build group 3
  x, o = group(x, blocks)
  aux += o

  # set block spec for group 4
  blocks = [((192,),  (96, 208), (16, 48), (64,)),
            None,
            ((160,), (112, 224), (24, 64), (64,)),
            ((128,), (128, 256), (24, 64), (64,)),
            ((112,), (144, 288), (32, 64), (64,)),
            None,
            ((256,), (160, 320), (32, 128), (128,))]

  # build group 4
  x, o = group(x, blocks, classes=classes)
  aux += o

  # set block specs for group 5
  blocks = [((256,), (160, 320), (32, 128), (128,)),
            ((384,), (192, 384), (48, 128), (128,))]
  # build group 5
  x, o = group(x, blocks, pooling=False)
  aux += o


  return x, aux

## The task component

In [43]:
def task(x, classes, dropout=0.4):
  '''
  Build the task component.

  x       : input to the task component
  classes : nr. of output classes
  dropout : dropout rate
  '''

  # a 7x7 average pooling
  x = keras.layers.AveragePooling2D(pool_size=7)(x)

  # flatten the output from the avg pooling
  x = keras.layers.Flatten()(x)

  # dropout for regularization
  x = keras.layers.Dropout(rate=dropout)(x)

  # softmax layer
  outputs = keras.layers.Dense(units=classes,
                               activation="softmax",
                               kernel_initializer="glorot_uniform")(x)

  return outputs

## Inception model v1

In [44]:
def build_model(shape=(224, 224, 3), dropout=0.4):

  # define inputs
  inputs = keras.layers.Input(shape=shape)

  # the stem component
  x = stem(inputs)

  # the learner component
  x, aux = learner(x, 1000)

  # the task component
  outputs = task(x, 1000, dropout)

  # build the model
  model = keras.Model(inputs=inputs,
                      outputs=[outputs] + aux,
                      name="inception_v1")

  return model

In [45]:
# create the inception model (v1)
inception_model_v1 = build_model()

# print a summary
inception_model_v1.summary()

Model: "inception_v1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_11 (ZeroPadding  (None, 230, 230, 3)  0          ['input_5[0][0]']                
 2D)                                                                                              
                                                                                                  
 conv2d_11 (Conv2D)             (None, 112, 112, 64  9472        ['zero_padding2d_11[0][0]']      
                                )                                                      

# Inception v2

## The stem component

In [3]:
def stem(inputs):
  '''
  Build the stem component of Inception v1.
  It extracts the coarse-level features of the input.

  input: the input image
  '''

  # apply ZeroPadding so that the size of the input is preserved
  x = keras.layers.ZeroPadding2D(padding=3)(inputs)

  # apply a strided 7x7 convolution to extract features
  # it will generate 64 output feature maps
  x = keras.layers.Conv2D(filters=64,
                          kernel_size=7,
                          strides=2,
                          padding="valid",
                          use_bias=False,
                          kernel_initializer="glorot_uniform"
                          )(x)

  # a BatchNormalization layer followed by a ReLu activation layer
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # again, apply ZeroPadding
  x = keras.layers.ZeroPadding2D(padding=1)(x)

  # apply strided max pooling with a 3x3 window
  # the size of each feature map will be reduced by 75%
  x = keras.layers.MaxPooling2D(pool_size=3, strides=2)(x)

  # apply a 1x1 convolution
  x = keras.layers.Conv2D(filters = 64,
                          kernel_size=1,
                          padding="same",
                          use_bias = False,
                          kernel_initializer="glorot_uniform")(x)

  # a BatchNormalization layer followed by a ReLu activation layer
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # zero-padding
  x = keras.layers.ZeroPadding2D(padding=1)(x)

  # a 3x3 convolution; it will output 192 feature maps
  x = keras.layers.Conv2D(filters=192,
                          kernel_size=3,
                          strides=1,
                          padding="valid",
                          use_bias=False,
                          kernel_initializer="glorot_uniform"
                          )(x)

  # a BatchNormalization layer followed by a ReLu activation layer
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # zero-padding
  x = keras.layers.ZeroPadding2D(padding=1)(x)

  # apply strided max pooling with a 3x3 window
  # the size of each feature map will be reduced by 75%
  x = keras.layers.MaxPooling2D(pool_size=3, strides=2)(x)


  return x

## The learner component

### Auxiliary Classifier

In [4]:
def auxiliary(x, classes):
  '''
  Build the auxiliary classifier.

  x       : input to the auxiliary classifier
  classes : number of output classes
  '''

  # strided average pooling operation with a 5x5 window
  x = keras.layers.AveragePooling2D(pool_size=5, strides=3)(x)

  # a 1x1 convolution which will output 128 feature maps
  x = keras.layers.Conv2D(filters=128,
                          kernel_size=1,
                          padding="same",
                          use_bias=False,
                          kernel_initializer="glorot_uniform"
                          )(x)

  # a BatchNormalization layer followed by a ReLU action layer
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # flatten the feature maps before passing them to the Dense layer
  x = keras.layers.Flatten()(x)

  # a fully connected layer with 1024 neurons
  x = keras.layers.Dense(units=1024,
                         activation="relu",
                         kernel_initializer="glorot_uniform")(x)

  # dropout layer for regularization
  x = keras.layers.Dropout(rate=0.7)(x)

  # softmax layer to output probabilities for each class
  output = keras.layers.Dense(units=classes,
                              activation="softmax",
                              kernel_initializer="glorot_uniform")(x)


  return output

### Inception Block

In [5]:
def inception_block(input, filters_1x1, filters_3x3, filters_5x5, filter_pool):
  '''
  Build the inception block/module.

  input : input to the inception block/module
  filters_1x1 : nr. of filters for the 1x1 conv branch of the inception module
  filters_3x3 : nr. of filters for the 3x3 conv branch of the inception model
  filters_5x5 : nr. of filters for the 5x5 conv branch of the inception module
  filters_pool: nr. of filters for the max pooling branch of the inception module
  '''

  # create the 1x1 conv branch with a BatchNormalization followed by a ReLU activation
  branch_1x1 = keras.layers.Conv2D(filters=filters_1x1[0],
                                   kernel_size=1,
                                   strides=1,
                                   padding="same",
                                   use_bias=False,
                                   kernel_initializer='glorot_uniform')(input)

  branch_1x1 = keras.layers.BatchNormalization()(branch_1x1)
  branch_1x1 = keras.layers.ReLU()(branch_1x1)

  # create the 3x3 conv branch with the 1x1 bottleneck convolution that performs dimensionality reduction
  branch_3x3 = keras.layers.Conv2D(filters=filters_3x3[0],
                                   kernel_size=1,
                                   strides=1,
                                   padding="same",
                                   use_bias=False,
                                   kernel_initializer='glorot_uniform')(input)

  # a BatchNormalization layer followed by a ReLU action layer
  branch_3x3 = keras.layers.BatchNormalization()(branch_3x3)
  branch_3x3 = keras.layers.ReLU()(branch_3x3)

  # zero-padding
  branch_3x3 = keras.layers.ZeroPadding2D(padding=1)(branch_3x3)

  # a 3x3 conv layer for feature extraction
  branch_3x3 = keras.layers.Conv2D(filters=filters_3x3[1],
                                   kernel_size=3,
                                   strides=1,
                                   padding="valid",
                                   use_bias=False,
                                   kernel_initializer='glorot_uniform')(branch_3x3)

  # a BatchNormalization layer followed by a ReLU activation layer
  branch_3x3 = keras.layers.BatchNormalization()(branch_3x3)
  branch_3x3 = keras.layers.ReLU()(branch_3x3)


  # create the 5x5 conv branch with the 1x1 bottleneck convolution that performs dimensionality reduction
  branch_5x5 = keras.layers.Conv2D(filters=filters_5x5[0],
                                   kernel_size=1,
                                   strides=1,
                                   padding="same",
                                   use_bias=False,
                                   kernel_initializer='glorot_uniform')(input)

  # a BatchNormalization layer followed by a ReLU activation layer
  branch_5x5 = keras.layers.BatchNormalization()(branch_5x5)
  branch_5x5 = keras.layers.ReLU()(branch_5x5)

  # zero-padding
  branch_5x5 = keras.layers.ZeroPadding2D(padding=1)(branch_5x5)

  # a 3x3 convolution layer
  branch_5x5 = keras.layers.Conv2D(filters=filters_5x5[1],
                                   kernel_size=3,
                                   strides=1,
                                   padding="valid",
                                   use_bias=False,
                                   kernel_initializer='glorot_uniform')(branch_5x5)

  # a BatchNormalization layer followed by a ReLU activation layer
  branch_5x5 = keras.layers.BatchNormalization()(branch_5x5)
  branch_5x5 = keras.layers.ReLU()(branch_5x5)


  # create the max pooling branch with 1x1 linear projection for dimensionality expansion
  branch_pool = keras.layers.MaxPooling2D(pool_size=3,
                                          strides=1,
                                          padding="same")(input)

  # a 1x1 convolution layer
  branch_pool = keras.layers.Conv2D(filters=filter_pool[0],
                                    kernel_size=1,
                                    strides=1,
                                    padding='same',
                                    use_bias=False,
                                    kernel_initializer='glorot_uniform')(branch_pool)
  # a BatchNormalization layer followed by a ReLU activation layer
  branch_pool = keras.layers.BatchNormalization()(branch_pool)
  branch_pool = keras.layers.ReLU()(branch_pool)

  # Concatenate the output feature maps of the branches
  x = keras.layers.Concatenate()([branch_1x1, branch_3x3, branch_5x5, branch_pool])

  return x


### Inception Group

In [6]:
def group(x, blocks, pooling=True, classes=1000):
  '''
  Build an Inception group.

  x       : input to the inception group
  blocks  : list of tuples containing nr. of filters for each block in the group
  pooling : flag indicating whether to end the group with max pooling
  classes : nr. of classes for auxiliary classifier
  '''

  # list to save the auxiliary outputs
  aux = []

  # build the inception blocks
  for block in blocks:

    # adding the auxiliary classifier
    if block is None:
      # save the auxiliary output
      aux.append(auxiliary(x,classes))

    else:
       # create inception block
       x = inception_block(x,
                           block[0],
                           block[1],
                           block[2],
                           block[3])


  # if pooling is True
  if pooling:
    # zero-padding
    x = keras.layers.ZeroPadding2D(padding=(1,1))(x)

    # max pooling
    x = keras.layers.MaxPooling2D(pool_size=3, strides=2)(x)


  return x, aux

Finally, we can build the learner component as follows:

In [7]:
def learner(x, classes):
  """ Build the learner component.

      x        : input to the learner
      classes: number of output classes
  """

  # a list to keep track of the auxiliary classifiers
  aux = []

  # the block specs for group 3
  blocks = [((64,),  (96,128),   (16, 32), (32,)),
              ((128,), (128, 192), (32, 96), (64,))]
  # build group 3
  x, o = group(x, blocks)
  aux += o

  # the block specs for group 4
  blocks = [((192,),  (96, 208), (16, 48), (64,)),
            None,
            ((160,), (112, 224), (24, 64), (64,)),
            ((128,), (128, 256), (24, 64), (64,)),
            ((112,), (144, 288), (32, 64), (64,)),
            None,
            ((256,), (160, 320), (32, 128), (128,))]
  # build group 4
  x, o = group(x, blocks, classes=classes)
  aux += o

  # the block specs for group 5
  blocks = [((256,), (160, 320), (32, 128), (128,)),
            ((384,), (192, 384), (48, 128), (128,))]
  # build group 5
  x, o = group(x, blocks, pooling=False)
  aux += o


  return x, aux

## The task component

In [8]:
def task(x, classes, dropout=0.4):
  '''
  Build the task component.

  x       : input to the task component
  classes : nr. of output classes
  dropout : dropout rate
  '''

  # a 7x7 average pooling
  x = keras.layers.AveragePooling2D(pool_size=7)(x)

  # flatten the output from the avg pooling
  x = keras.layers.Flatten()(x)

  # dropout for regularization
  x = keras.layers.Dropout(rate=dropout)(x)

  # softmax layer
  outputs = keras.layers.Dense(units=classes,
                               activation="softmax",
                               kernel_initializer="glorot_uniform")(x)

  return outputs

## Inception model v2

In [9]:
def build_model(shape=(224, 224, 3), dropout=0.4):

  # define inputs
  inputs = keras.layers.Input(shape=shape)

  # the stem component
  x = stem(inputs)

  # the learner component
  x, aux = learner(x, 1000)

  # the task component
  outputs = task(x, 1000, dropout)

  # build the model
  model = keras.Model(inputs=inputs,
                      outputs=[outputs] + aux,
                      name="inception_v2")

  return model

In [11]:
# create the inception model (v1)
inception_model_v2 = build_model()

# print a summary
inception_model_v2.summary()

Model: "inception_v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_24 (ZeroPadding  (None, 230, 230, 3)  0          ['input_2[0][0]']                
 2D)                                                                                              
                                                                                                  
 conv2d_59 (Conv2D)             (None, 112, 112, 64  9408        ['zero_padding2d_24[0][0]']      
                                )                                                      